In [48]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.proportion import proportions_ztest

# Задача
Оцените результаты эксперимента:
1. Стат значимо ли отличается метрика конверсии в двух группах? 
2. Какие рекомендации дадите менеджеру?

In [49]:
df = pd.read_csv('ab_test.csv', sep=',')
df.head()

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         294478 non-null  int64 
 1   time       294478 non-null  object
 2   con_treat  294478 non-null  object
 3   page       294478 non-null  object
 4   converted  294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


### Избавимся от некорректных строк

In [51]:
df.id.nunique() # yникальные id

290584

In [52]:
df = df[(df['con_treat'] == 'control')&(df['page'] == 'old_page')|(df['con_treat'] == 'treatment')&(df['page'] == 'new_page')]
df.shape

(290585, 5)

### Анализ данных

In [53]:
df.describe()

,id,converted
count,290585.000000,290585.000000
mean,788004.825246,0.119597
std,91224.582639,0.324490
min,630000.000000,0.000000
25%,709035.000000,0.000000
50%,787995.000000,0.000000
75%,866956.000000,0.000000
max,945999.000000,1.000000


In [54]:
df.con_treat.value_counts()

con_treat
treatment    145311
control      145274
Name: count, dtype: int64

In [55]:
n_cont = df[df['con_treat'] == 'control'].shape[0]
n_test = df[df['con_treat'] == 'treatment'].shape[0]

k_cont = df[df['con_treat'] == 'control'].converted.sum()
k_test = df[df['con_treat'] == 'treatment'].converted.sum()


print(k_cont, k_test)
print(n_cont, n_test)

17489 17264
145274 145311


In [56]:
# Хи-квадрат
chisq, pvalue, table = proportions_chisquare(np.array([k_cont, k_test]), np.array([n_cont, n_test]))
print('Результаты:','chisq = %.3f, pvalue = %.3f'%(chisq, pvalue))

if abs(pvalue) > 0.05:
   print('Нулевая гипотеза - Между конверсией в двух группах нет статистически значимых различий')
else:
   print('Альтернативная гипотеза – Между конверсией в двух группах есть статистически значимые различия')

Результаты: chisq = 1.720, pvalue = 0.190
Нулевая гипотеза - Между конверсией в двух группах нет статистически значимых различий


In [58]:
# Z-test -> проверкa
z_score, z_pvalue = proportions_ztest(np.array([k_cont, k_test]), np.array([n_cont, n_test]))
print('Результаты:','z_score = %.3f, pvalue = %.3f'%(z_score, z_pvalue))

if abs(z_pvalue) > 0.05:
   print('Нулевая гипотеза - Между конверсией в двух группах нет статистически значимых различий')
else:
   print('Альтернативная гипотеза – Между конверсией в двух группах есть статистически значимые различия')

Результаты: z_score = 1.312, pvalue = 0.190
Нулевая гипотеза - Между конверсией в двух группах нет статистически значимых различий


# Результат
1. Нет созначимых различий. 
2. Ничего не меняем. Нет смысла выкатывать новую версию.